In [ ]:
!pip install pyshark pytorch-tabnet catboost lightgbm scapy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.5/44.5 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 45.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 24.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/2

In [ ]:
# Import necessary libraries
import pyshark
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from pytorch_tabnet.tab_model import TabNetClassifier
import catboost as cb
import lightgbm as lgb
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from scapy.all import *
import pyshark

In [ ]:
# Install required dependencies
!pip install pyshark nest_asyncio
!apt-get update
!apt-get install -y wireshark tshark  # Install Wireshark and TShark
!yes | apt-get install -y tshark  # Automatically accept prompts



Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,375 kB]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,748 kB]
Get:9 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,692 kB]
Get:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [3,798 kB]
Get:13 https://ppa.la

In [ ]:
# Import libraries
import nest_asyncio
nest_asyncio.apply()  # Allow nested event loops
import pyshark
import pandas as pd

In [ ]:
# Step 1: Load network traffic data
# Replace 'traffic.pcap' with your PCAP file name after uploading it.
cap = pyshark.FileCapture('/content/test.pcap')

In [ ]:
# Step 2: Feature Extraction
packet_data = []
for packet in cap:
    try:
        protocol = packet.highest_layer
        src_ip = packet.ip.src
        dst_ip = packet.ip.dst
        src_port = packet[packet.transport_layer].srcport
        dst_port = packet[packet.transport_layer].dstport
        packet_size = int(packet.length)
        time_stamp = float(packet.sniff_timestamp)
        packet_data.append([protocol, src_ip, dst_ip, src_port, dst_port, packet_size, time_stamp])
    except AttributeError:
        continue  # Skip packets without required attributes

# Step 3: Save to DataFrame and CSV
columns = ['protocol', 'src_ip', 'dst_ip', 'src_port', 'dst_port', 'packet_size', 'time_stamp']
df = pd.DataFrame(packet_data, columns=columns)
df.to_csv('extracted_traffic_data.csv', index=False)
print("Extracted data saved to 'extracted_traffic_data.csv'")
print(df.head())  # Display first few rows for verification

Extracted data saved to 'extracted_traffic_data.csv'
  protocol         src_ip         dst_ip src_port dst_port  packet_size  \
0      TCP  192.168.100.9   20.192.44.80    61039      443           54   
1      TLS    5.78.100.21  192.168.100.9      443    61137        14174   
2      TCP  192.168.100.9    5.78.100.21    61137      443           54   
3      TLS    5.78.100.21  192.168.100.9      443    61137         1466   
4      TCP  192.168.100.9    5.78.100.21    61137      443           54   

     time_stamp  
0  1.742142e+09  
1  1.742142e+09  
2  1.742142e+09  
3  1.742142e+09  
4  1.742142e+09  


In [ ]:
# Create DataFrame
columns = ['protocol', 'src_ip', 'dst_ip', 'src_port', 'dst_port', 'packet_size', 'time_stamp']
df = pd.DataFrame(packet_data, columns=columns)

In [ ]:
# Step 3: Feature Engineering
df['time_interval'] = df['time_stamp'].diff().fillna(0)
df['src_ip_encoded'] = df['src_ip'].apply(lambda x: int(x.split('.')[0]))
df['dst_ip_encoded'] = df['dst_ip'].apply(lambda x: int(x.split('.')[0]))
df = df.drop(['src_ip', 'dst_ip'], axis=1)

# Simulate labels (replace with your actual labels)
df['label'] = np.random.randint(0, 2, df.shape[0])

# Save raw extracted data to CSV
df.to_csv('extracted_traffic_data.csv', index=False)
print("Extracted data saved to 'extracted_traffic_data.csv'")

Extracted data saved to 'extracted_traffic_data.csv'


In [ ]:
# Step 4: Data Preprocessing
categorical_cols = ['protocol', 'src_port', 'dst_port']
numerical_cols = ['packet_size', 'time_interval', 'src_ip_encoded', 'dst_ip_encoded']

numerical_transformer = StandardScaler()
categorical_transformer = OneHotEncoder(handle_unknown='ignore', sparse_output=False)  # Fixed here

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

X = df.drop('label', axis=1)
y = df['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train_processed = preprocessor.fit_transform(X_train)
X_test_processed = preprocessor.transform(X_test)

# Save preprocessed data
pd.DataFrame(X_train_processed).to_csv('X_train_processed.csv', index=False)
pd.DataFrame(X_test_processed).to_csv('X_test_processed.csv', index=False)
print("Preprocessed data saved to 'X_train_processed.csv' and 'X_test_processed.csv'")

Preprocessed data saved to 'X_train_processed.csv' and 'X_test_processed.csv'


In [ ]:
# Step 5: Train Machine Learning Models
# Model 1: TabNet
tabnet_model = TabNetClassifier(n_d=8, n_a=8, n_steps=3, verbose=1)
tabnet_model.fit(
    X_train_processed, y_train,
    eval_set=[(X_test_processed, y_test)],
    max_epochs=20,
    patience=5
)

# Model 2: CatBoost
catboost_model = cb.CatBoostClassifier(iterations=100, depth=6, learning_rate=0.1, verbose=0)
catboost_model.fit(X_train_processed, y_train)

# Model 3: LightGBM
lgbm_model = lgb.LGBMClassifier(n_estimators=100, max_depth=6, learning_rate=0.1)
lgbm_model.fit(X_train_processed, y_train)

# Model 4: Stacking Ensemble
estimators = [
    ('tabnet', tabnet_model),
    ('catboost', catboost_model),
    ('lightgbm', lgbm_model)
]
stacking_model = StackingClassifier(
    estimators=estimators,
    final_estimator=LogisticRegression(max_iter=1000),
    cv=5
)
stacking_model.fit(X_train_processed, y_train)

/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")
/usr/local/lib/python3.11/dist-packages/torch/onnx/symbolic_caffe2.py:6: RuntimeWarning: coroutine 'Capture._get_tshark_process' was never awaited
  from torch.onnx import symbolic_helper, symbolic_opset9 as opset9


epoch 0  | loss: 1.06025 | val_0_auc: 0.47326 |  0:00:01s
epoch 1  | loss: 0.87017 | val_0_auc: 0.50734 |  0:00:01s
epoch 2  | loss: 0.86791 | val_0_auc: 0.52097 |  0:00:01s
epoch 3  | loss: 0.96392 | val_0_auc: 0.52276 |  0:00:01s
epoch 4  | loss: 0.91176 | val_0_auc: 0.52715 |  0:00:01s
epoch 5  | loss: 0.86938 | val_0_auc: 0.50402 |  0:00:01s
epoch 6  | loss: 0.92148 | val_0_auc: 0.49654 |  0:00:01s
epoch 7  | loss: 0.89385 | val_0_auc: 0.49733 |  0:00:01s
epoch 8  | loss: 0.7952  | val_0_auc: 0.49712 |  0:00:02s
epoch 9  | loss: 0.79136 | val_0_auc: 0.50269 |  0:00:02s

Early stopping occurred at epoch 9 with best_epoch = 4 and best_val_0_auc = 0.52715


/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")
/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/abstract_model.py:687: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)


[LightGBM] [Info] Number of positive: 780, number of negative: 752
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000844 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 402
[LightGBM] [Info] Number of data points in the train set: 1532, number of used features: 20
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.509138 -> initscore=0.036558
[LightGBM] [Info] Start training from score 0.036558
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")
/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")
/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/abstract_model.py:687: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)


...

epoch 0  | loss: 1.08161 |  0:00:00s
epoch 1  | loss: 0.96767 |  0:00:00s
epoch 2  | loss: 0.91649 |  0:00:00s
epoch 3  | loss: 0.90108 |  0:00:00s
epoch 4  | loss: 0.87639 |  0:00:00s
epoch 5  | loss: 0.91633 |  0:00:00s
epoch 6  | loss: 0.88851 |  0:00:00s
epoch 7  | loss: 0.8819  |  0:00:00s
epoch 8  | loss: 0.84544 |  0:00:00s
epoch 9  | loss: 0.817   |  0:00:00s
epoch 10 | loss: 0.84357 |  0:00:00s
epoch 11 | loss: 0.83428 |  0:00:00s
epoch 12 | loss: 0.83112 |  0:00:00s
epoch 13 | loss: 0.82738 |  0:00:00s
epoch 14 | loss: 0.76042 |  0:00:00s
epoch 15 | loss: 0.7545  |  0:00:00s
epoch 16 | loss: 0.78693 |  0:00:00s
epoch 17 | loss: 0.74756 |  0:00:00s
epoch 18 | loss: 0.74859 |  0:00:00s
epoch 19 | loss: 0.7616  |  0:00:00s
epoch 20 | loss: 0.74176 |  0:00:00s
epoch 21 | loss: 0.75735 |  0:00:00s
epoch 22 | loss: 0.72893 |  0:00:00s
epoch 23 | loss: 0.73414 |  0:00:01s
epoch 24 | loss: 0.71643 |  0:00:01s
epoch 25 | loss: 0.71038 |  0:00:01s
epoch 26 | loss: 0.72824 |  0:00:01s
e

/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")
/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/abstract_model.py:687: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)


epoch 1  | loss: 0.95914 |  0:00:00s
epoch 2  | loss: 0.97855 |  0:00:00s
epoch 3  | loss: 0.85789 |  0:00:00s
epoch 4  | loss: 0.90535 |  0:00:00s
epoch 5  | loss: 0.8632  |  0:00:00s
epoch 6  | loss: 0.81874 |  0:00:00s
epoch 7  | loss: 0.8189  |  0:00:00s
epoch 8  | loss: 0.78418 |  0:00:00s
epoch 9  | loss: 0.78743 |  0:00:00s
epoch 10 | loss: 0.74643 |  0:00:00s
epoch 11 | loss: 0.77644 |  0:00:00s
epoch 12 | loss: 0.7721  |  0:00:00s
epoch 13 | loss: 0.76626 |  0:00:00s
epoch 14 | loss: 0.73303 |  0:00:00s
epoch 15 | loss: 0.74638 |  0:00:00s
epoch 16 | loss: 0.72457 |  0:00:00s
epoch 17 | loss: 0.75345 |  0:00:00s
epoch 18 | loss: 0.72716 |  0:00:00s
epoch 19 | loss: 0.74948 |  0:00:00s
epoch 20 | loss: 0.73032 |  0:00:00s
epoch 21 | loss: 0.72616 |  0:00:00s
epoch 22 | loss: 0.72419 |  0:00:00s
epoch 23 | loss: 0.72927 |  0:00:01s
epoch 24 | loss: 0.71419 |  0:00:01s
epoch 25 | loss: 0.71178 |  0:00:01s
epoch 26 | loss: 0.72441 |  0:00:01s
epoch 27 | loss: 0.71165 |  0:00:01s
e

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11

StackingClassifier(cv=5,
                   estimators=[('tabnet',
                                TabNetClassifier(n_d=8,
                                                 n_a=8,
                                                 n_steps=3,
                                                 gamma=1.3,
                                                 cat_idxs=[],
                                                 cat_dims=[],
                                                 cat_emb_dim=[],
                                                 n_independent=2,
                                                 n_shared=2,
                                                 epsilon=1e-15,
                                                 momentum=0.02,
                                                 lambda_sparse=0.001,
                                                 seed=0,
                                                 clip_value=1,
                                                 verbose=1,
                                                 optimizer_fn=<class 'torch.optim.adam.Adam'>,
                                                 optimizer_params={'lr': 0.02},
                                                 scheduler_fn=None,
                                                 scheduler_params={},
                                                 mask_type='sparsemax',
                                                 input_dim=137,
                                                 output_dim=2,
                                                 device_name='auto',
                                                 n_shared_decoder=1,
                                                 n_indep_decoder=1,
                                                 grouped_features=[])),
                               ('catboost',
                                <catboost.core.CatBoostClassifier object at 0x7c6608ec6410>),
                               ('lightgbm', LGBMClassifier(max_depth=6))],
                   final_estimator=LogisticRegression(max_iter=1000))

In [ ]:
# Step 6: Evaluate Models and Save Results
models = {
    'TabNet': tabnet_model,
    'CatBoost': catboost_model,
    'LightGBM': lgbm_model,
    'Stacking Ensemble': stacking_model
}

evaluation_results = []

for name, model in models.items():
    y_pred = model.predict(X_test_processed)
    y_prob = model.predict_proba(X_test_processed)[:, 1] if hasattr(model, 'predict_proba') else None

    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')
    roc_auc = roc_auc_score(y_test, y_prob) if y_prob is not None else 'N/A'

    # Save predictions to CSV
    pd.DataFrame({'y_true': y_test, 'y_pred': y_pred, 'y_prob': y_prob if y_prob is not None else 'N/A'}).to_csv(f'{name}_predictions.csv', index=False)

    # Collect evaluation metrics
    evaluation_results.append([name, accuracy, precision, recall, f1, roc_auc])

# Save evaluation metrics to CSV
eval_df = pd.DataFrame(evaluation_results, columns=['Model', 'Accuracy', 'Precision', 'Recall', 'F1-Score', 'ROC-AUC'])
eval_df.to_csv('model_evaluation_results.csv', index=False)
print("Model evaluation results saved to 'model_evaluation_results.csv'")

# Display results
print(eval_df)

Model evaluation results saved to 'model_evaluation_results.csv'
               Model  Accuracy  Precision    Recall  F1-Score   ROC-AUC
0             TabNet  0.496084   0.246099  0.496084  0.328991  0.527148
1           CatBoost  0.483029   0.480798  0.483029  0.474246  0.480938
2           LightGBM  0.503916   0.503957  0.503916  0.503923  0.507158
3  Stacking Ensemble  0.543081   0.543441  0.543081  0.540542  0.517712


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_al

In [ ]:
# # Install dependencies
# !pip install pyshark nest_asyncio pytorch-tabnet catboost lightgbm scapy
# !apt-get update
# !apt-get install -y wireshark tshark
# !yes | apt-get install -y tshark

# Import libraries
import nest_asyncio
nest_asyncio.apply()
import pyshark
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from pytorch_tabnet.tab_model import TabNetClassifier
import catboost as cb
import lightgbm as lgb
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression

# Load PCAP
cap = pyshark.FileCapture('/content/test.pcap')

# Feature Extraction
packet_data = []
for packet in cap:
    try:
        protocol = packet.highest_layer
        src_ip = packet.ip.src
        dst_ip = packet.ip.dst
        src_port = packet[packet.transport_layer].srcport
        dst_port = packet[packet.transport_layer].dstport
        packet_size = int(packet.length)
        time_stamp = float(packet.sniff_timestamp)
        packet_data.append([protocol, src_ip, dst_ip, src_port, dst_port, packet_size, time_stamp])
    except AttributeError:
        continue

# Create DataFrame
columns = ['protocol', 'src_ip', 'dst_ip', 'src_port', 'dst_port', 'packet_size', 'time_stamp']
df = pd.DataFrame(packet_data, columns=columns)
df['time_interval'] = df['time_stamp'].diff().fillna(0)
df['src_ip_encoded'] = df['src_ip'].apply(lambda x: int(x.split('.')[0]))
df['dst_ip_encoded'] = df['dst_ip'].apply(lambda x: int(x.split('.')[0]))
df = df.drop(['src_ip', 'dst_ip'], axis=1)

# Replace with real labels (example placeholder)
# TODO: Load your labeled data here, e.g., from a CSV or manual annotation
df['label'] = np.random.randint(0, 2, df.shape[0])  # TEMPORARY: Replace this!
df.to_csv('extracted_traffic_data.csv', index=False)
print("Extracted data saved to 'extracted_traffic_data.csv'")

# Preprocessing
categorical_cols = ['protocol', 'src_port', 'dst_port']
numerical_cols = ['packet_size', 'time_interval', 'src_ip_encoded', 'dst_ip_encoded']

numerical_transformer = StandardScaler()
categorical_transformer = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

X = df.drop('label', axis=1)
y = df['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train_processed = preprocessor.fit_transform(X_train)
X_test_processed = preprocessor.transform(X_test)

pd.DataFrame(X_train_processed).to_csv('X_train_processed.csv', index=False)
pd.DataFrame(X_test_processed).to_csv('X_test_processed.csv', index=False)
print("Preprocessed data saved to 'X_train_processed.csv' and 'X_test_processed.csv'")

# Train Models
tabnet_model = TabNetClassifier(n_d=8, n_a=8, n_steps=3, verbose=1)
tabnet_model.fit(X_train_processed, y_train, eval_set=[(X_test_processed, y_test)], max_epochs=20, patience=5)

catboost_model = cb.CatBoostClassifier(iterations=100, depth=6, learning_rate=0.1, verbose=0)
catboost_model.fit(X_train_processed, y_train)

lgbm_model = lgb.LGBMClassifier(n_estimators=100, max_depth=6, learning_rate=0.1)
lgbm_model.fit(X_train_processed, y_train)

estimators = [('tabnet', tabnet_model), ('catboost', catboost_model), ('lightgbm', lgbm_model)]
stacking_model = StackingClassifier(estimators=estimators, final_estimator=LogisticRegression(max_iter=1000), cv=5)
stacking_model.fit(X_train_processed, y_train)

# Evaluate Models
models = {'TabNet': tabnet_model, 'CatBoost': catboost_model, 'LightGBM': lgbm_model, 'Stacking': stacking_model}
evaluation_results = []

for name, model in models.items():
    y_pred = model.predict(X_test_processed)
    y_prob = model.predict_proba(X_test_processed)[:, 1] if hasattr(model, 'predict_proba') else None

    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted', zero_division=0)  # Handle undefined precision
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')
    roc_auc = roc_auc_score(y_test, y_prob, ensure_all_finite=True) if y_prob is not None else 'N/A'  # Updated parameter

    pd.DataFrame({'y_true': y_test, 'y_pred': y_pred}).to_csv(f'{name}_predictions.csv', index=False)
    evaluation_results.append([name, accuracy, precision, recall, f1, roc_auc])

# Save and Display Results
eval_df = pd.DataFrame(evaluation_results, columns=['Model', 'Accuracy', 'Precision', 'Recall', 'F1-Score', 'ROC-AUC'])
eval_df.to_csv('model_evaluation_results.csv', index=False)
print("Evaluation results saved to 'model_evaluation_results.csv'")
print(eval_df)

Extracted data saved to 'extracted_traffic_data.csv'
Preprocessed data saved to 'X_train_processed.csv' and 'X_test_processed.csv'
epoch 0  | loss: 1.06411 | val_0_auc: 0.53255 |  0:00:00s


/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


epoch 1  | loss: 0.93415 | val_0_auc: 0.51921 |  0:00:00s
epoch 2  | loss: 0.97527 | val_0_auc: 0.5007  |  0:00:00s
epoch 3  | loss: 0.89972 | val_0_auc: 0.50899 |  0:00:00s
epoch 4  | loss: 0.90677 | val_0_auc: 0.50857 |  0:00:00s
epoch 5  | loss: 0.86346 | val_0_auc: 0.50477 |  0:00:00s

Early stopping occurred at epoch 5 with best_epoch = 0 and best_val_0_auc = 0.53255


/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")
/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/abstract_model.py:687: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)


[LightGBM] [Info] Number of positive: 770, number of negative: 762
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000048 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 402
[LightGBM] [Info] Number of data points in the train set: 1532, number of used features: 20
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.502611 -> initscore=0.010444
[LightGBM] [Info] Start training from score 0.010444
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


...

/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")
/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/abstract_model.py:687: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)


epoch 0  | loss: 1.03449 |  0:00:00s
epoch 1  | loss: 0.99943 |  0:00:00s
epoch 2  | loss: 0.89527 |  0:00:00s
epoch 3  | loss: 0.94223 |  0:00:00s
epoch 4  | loss: 0.91376 |  0:00:00s
epoch 5  | loss: 0.83336 |  0:00:00s
epoch 6  | loss: 0.84435 |  0:00:00s
epoch 7  | loss: 0.80075 |  0:00:00s
epoch 8  | loss: 0.7802  |  0:00:00s
epoch 9  | loss: 0.78231 |  0:00:00s
epoch 10 | loss: 0.80073 |  0:00:00s
epoch 11 | loss: 0.78042 |  0:00:00s
epoch 12 | loss: 0.79316 |  0:00:00s
epoch 13 | loss: 0.79137 |  0:00:00s
epoch 14 | loss: 0.7739  |  0:00:00s
epoch 15 | loss: 0.77705 |  0:00:00s
epoch 16 | loss: 0.75735 |  0:00:01s
epoch 17 | loss: 0.78175 |  0:00:01s
epoch 18 | loss: 0.77139 |  0:00:01s
epoch 19 | loss: 0.761   |  0:00:01s
epoch 20 | loss: 0.75496 |  0:00:01s
epoch 21 | loss: 0.74396 |  0:00:01s
epoch 22 | loss: 0.74374 |  0:00:01s
epoch 23 | loss: 0.73234 |  0:00:01s
epoch 24 | loss: 0.74142 |  0:00:01s
epoch 25 | loss: 0.73096 |  0:00:01s
epoch 26 | loss: 0.7302  |  0:00:01s
e

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11

[LightGBM] [Info] Number of positive: 616, number of negative: 609
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000062 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 384
[LightGBM] [Info] Number of data points in the train set: 1225, number of used features: 18
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.502857 -> initscore=0.011429
[LightGBM] [Info] Start training from score 0.011429
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


TypeError: got an unexpected keyword argument 'ensure_all_finite'

In [ ]:
!pip install pandas numpy pyshark scikit-learn pytorch-tabnet catboost lightgbm torch

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import pyshark
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.linear_model import LogisticRegression
from pytorch_tabnet.tab_model import TabNetClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import StackingClassifier
import torch  # Crucial import to fix the NameError

# 1. Extract Data from PCAP
def extract_packet_data(pcap_file):
    """Extract features from a PCAP file using pyshark."""
    cap = pyshark.FileCapture(pcap_file)
    packet_data = []

    for packet in cap:
        try:
            protocol = packet.highest_layer
            src_ip = packet.ip.src if 'ip' in packet else None
            dst_ip = packet.ip.dst if 'ip' in packet else None
            src_port = packet[packet.transport_layer].srcport if packet.transport_layer else None
            dst_port = packet[packet.transport_layer].dstport if packet.transport_layer else None
            packet_size = int(packet.length)
            time_stamp = float(packet.sniff_timestamp)
            tcp_flags = packet.tcp.flags if 'tcp' in packet else None

            packet_data.append([protocol, src_ip, dst_ip, src_port, dst_port, packet_size, time_stamp, tcp_flags])
        except AttributeError:
            continue

    cap.close()
    return pd.DataFrame(packet_data, columns=['protocol', 'src_ip', 'dst_ip', 'src_port', 'dst_port',
                                              'packet_size', 'time_stamp', 'tcp_flags'])

# 2. Preprocess Data
def preprocess_data(df):
    """Preprocess the extracted packet data."""
    # Handle missing values
    df = df.fillna({'src_port': 0, 'dst_port': 0, 'tcp_flags': 'None'})

    # Convert categorical features to numeric
    df['protocol'] = df['protocol'].astype('category').cat.codes
    df['tcp_flags'] = df['tcp_flags'].astype('category').cat.codes

    # Drop IP addresses (or encode if relevant)
    df = df.drop(['src_ip', 'dst_ip'], axis=1)

    # Feature engineering: time interval between packets
    df['time_interval'] = df['time_stamp'].diff().fillna(0)
    df = df.drop('time_stamp', axis=1)

    return df

# 3. Simulate Labels (Replace with real labels if available)
def simulate_labels(df):
    """Simulate labels for demonstration; replace with real labels if available."""
    # Example: Label packets with size > 95th percentile as anomalies
    threshold = df['packet_size'].quantile(0.95)
    df['label'] = (df['packet_size'] > threshold).astype(int)
    return df

# 4. Main Execution
if __name__ == "__main__":
    # Load and preprocess data
    pcap_file = 'test.pcap'  # Replace with your actual PCAP file path
    df = extract_packet_data(pcap_file)
    df = preprocess_data(df)
    df = simulate_labels(df)  # Replace this with real labels if you have them

    # Save extracted data
    df.to_csv('extracted_traffic_data.csv', index=False)

    # Split features and labels
    X = df.drop('label', axis=1)
    y = df['label']

    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Scale features
    scaler = StandardScaler()
    X_train_processed = scaler.fit_transform(X_train)
    X_test_processed = scaler.transform(X_test)

    # Save preprocessed data
    pd.DataFrame(X_train_processed, columns=X.columns).to_csv('X_train_processed.csv', index=False)
    pd.DataFrame(X_test_processed, columns=X.columns).to_csv('X_test_processed.csv', index=False)

    # Define base models
    tabnet = TabNetClassifier(
        n_d=8, n_a=8, n_steps=3, gamma=1.3, n_independent=2, n_shared=2,
        cat_idxs=[], cat_dims=[], cat_emb_dim=1,
        lambda_sparse=1e-3, optimizer_params=dict(lr=2e-2),
        verbose=1, device_name='cuda' if torch.cuda.is_available() else 'cpu'
    )

    catboost = CatBoostClassifier(iterations=100, learning_rate=0.1, depth=6, verbose=0)
    lightgbm = LGBMClassifier(n_estimators=100, learning_rate=0.1, max_depth=6)

    # Define stacking ensemble
    base_models = [
        ('tabnet', tabnet),
        ('catboost', catboost),
        ('lightgbm', lightgbm)
    ]
    stacking = StackingClassifier(estimators=base_models, final_estimator=LogisticRegression(), cv=5)

    # Collect all models
    models = {
        'TabNet': tabnet,
        'CatBoost': catboost,
        'LightGBM': lightgbm,
        'Stacking': stacking
    }

    # Train and evaluate models
    evaluation_results = []
    for name, model in models.items():
        print(f"Training {name}...")
        if name == 'TabNet':
            model.fit(
                X_train_processed, y_train,
                eval_set=[(X_test_processed, y_test)],
                eval_metric=['auc'],
                max_epochs=100,
                patience=10,
                batch_size=256,
                virtual_batch_size=128
            )
        else:
            model.fit(X_train_processed, y_train)

        # Predict
        y_pred = model.predict(X_test_processed)
        y_prob = model.predict_proba(X_test_processed)[:, 1] if hasattr(model, 'predict_proba') else None

        # Evaluate
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred, average='weighted', zero_division=0)
        recall = recall_score(y_test, y_pred, average='weighted')
        f1 = f1_score(y_test, y_pred, average='weighted')
        roc_auc = roc_auc_score(y_test, y_prob) if y_prob is not None else 'N/A'

        # Save predictions
        pd.DataFrame({'y_true': y_test, 'y_pred': y_pred}).to_csv(f'{name}_predictions.csv', index=False)

        # Store results
        evaluation_results.append([name, accuracy, precision, recall, f1, roc_auc])
        print(f"{name} - Accuracy: {accuracy:.4f}, AUC: {roc_auc if y_prob is not None else 'N/A'}")

    # Display and save evaluation results
    results_df = pd.DataFrame(evaluation_results, columns=['Model', 'Accuracy', 'Precision', 'Recall', 'F1', 'ROC_AUC'])
    print("\nEvaluation Results:")
    print(results_df)
    results_df.to_csv('evaluation_results.csv', index=False)

/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


Training TabNet...
epoch 0  | loss: 0.4632  | val_0_auc: 1.0     |  0:00:00s
epoch 1  | loss: 0.07842 | val_0_auc: 0.98956 |  0:00:00s
epoch 2  | loss: 0.03244 | val_0_auc: 1.0     |  0:00:00s
epoch 3  | loss: 0.05618 | val_0_auc: 1.0     |  0:00:00s
epoch 4  | loss: 0.02907 | val_0_auc: 1.0     |  0:00:00s
epoch 5  | loss: 0.03258 | val_0_auc: 1.0     |  0:00:01s
epoch 6  | loss: 0.03475 | val_0_auc: 1.0     |  0:00:01s
epoch 7  | loss: 0.02762 | val_0_auc: 1.0     |  0:00:01s
epoch 8  | loss: 0.02177 | val_0_auc: 1.0     |  0:00:01s
epoch 9  | loss: 0.03633 | val_0_auc: 1.0     |  0:00:01s
epoch 10 | loss: 0.01025 | val_0_auc: 1.0     |  0:00:01s

Early stopping occurred at epoch 10 with best_epoch = 0 and best_val_0_auc = 1.0


/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


TabNet - Accuracy: 0.9767, AUC: 1.0
Training CatBoost...
CatBoost - Accuracy: 1.0000, AUC: 1.0
Training LightGBM...
[LightGBM] [Info] Number of positive: 75, number of negative: 1472
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000115 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 444
[LightGBM] [Info] Number of data points in the train set: 1547, number of used features: 6
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.048481 -> initscore=-2.976889
[LightGBM] [Info] Start training from score -2.976889
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] N

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")
/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/abstract_model.py:687: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)


...

/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")
/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/abstract_model.py:687: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)


epoch 2  | loss: 0.3763  |  0:00:00s
epoch 3  | loss: 0.25542 |  0:00:00s
epoch 4  | loss: 0.15652 |  0:00:00s
epoch 5  | loss: 0.1114  |  0:00:00s
epoch 6  | loss: 0.08203 |  0:00:00s
epoch 7  | loss: 0.05725 |  0:00:00s
epoch 8  | loss: 0.05141 |  0:00:00s
epoch 9  | loss: 0.0481  |  0:00:00s
epoch 10 | loss: 0.03796 |  0:00:00s
epoch 11 | loss: 0.03333 |  0:00:00s
epoch 12 | loss: 0.0274  |  0:00:00s
epoch 13 | loss: 0.02146 |  0:00:00s
epoch 14 | loss: 0.01801 |  0:00:00s
epoch 15 | loss: 0.01658 |  0:00:00s
epoch 16 | loss: 0.0197  |  0:00:00s
epoch 17 | loss: 0.02158 |  0:00:00s
epoch 18 | loss: 0.02652 |  0:00:00s
epoch 19 | loss: 0.0114  |  0:00:00s
epoch 20 | loss: 0.0194  |  0:00:00s
epoch 21 | loss: 0.02277 |  0:00:00s
epoch 22 | loss: 0.01763 |  0:00:00s
epoch 23 | loss: 0.01036 |  0:00:00s
epoch 24 | loss: 0.012   |  0:00:01s
epoch 25 | loss: 0.02525 |  0:00:01s
epoch 26 | loss: 0.02032 |  0:00:01s
epoch 27 | loss: 0.02829 |  0:00:01s
epoch 28 | loss: 0.02151 |  0:00:01s
e

/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")
/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/abstract_model.py:687: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)


epoch 0  | loss: 1.1303  |  0:00:00s
epoch 1  | loss: 0.73175 |  0:00:00s
epoch 2  | loss: 0.55752 |  0:00:00s
epoch 3  | loss: 0.38789 |  0:00:00s
epoch 4  | loss: 0.35747 |  0:00:00s
epoch 5  | loss: 0.22619 |  0:00:00s
epoch 6  | loss: 0.19081 |  0:00:00s
epoch 7  | loss: 0.12937 |  0:00:00s
epoch 8  | loss: 0.1119  |  0:00:00s
epoch 9  | loss: 0.10725 |  0:00:00s
epoch 10 | loss: 0.08603 |  0:00:00s
epoch 11 | loss: 0.08939 |  0:00:00s
epoch 12 | loss: 0.07014 |  0:00:00s
epoch 13 | loss: 0.0543  |  0:00:00s
epoch 14 | loss: 0.05432 |  0:00:00s
epoch 15 | loss: 0.0501  |  0:00:00s
epoch 16 | loss: 0.03997 |  0:00:00s
epoch 17 | loss: 0.04285 |  0:00:01s
epoch 18 | loss: 0.0481  |  0:00:01s
epoch 19 | loss: 0.03843 |  0:00:01s
epoch 20 | loss: 0.04619 |  0:00:01s
epoch 21 | loss: 0.04585 |  0:00:01s
epoch 22 | loss: 0.03971 |  0:00:01s
epoch 23 | loss: 0.04147 |  0:00:01s
epoch 24 | loss: 0.03073 |  0:00:01s
epoch 25 | loss: 0.0492  |  0:00:01s
epoch 26 | loss: 0.06669 |  0:00:01s
e

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11